# _PyEnzyme_ - Export template

#### Usage

- This template offers all functionalities to write information to a given EnzymeML document. 
- Simply reduce the template to your application-specific variables and map these to your own application.

------------------------------

In [ ]:
from pyenzyme.enzymeml.core import EnzymeMLDocument, Vessel, Protein, Reactant, EnzymeReaction, Creator, Replicate
from pyenzyme.enzymeml.models import KineticModel
from pyenzyme.enzymeml.tools import EnzymeMLWriter

## Initialize EnzymeML document

- The blank EnzymeMLDocument object serves as a container for all those objects to be added
- Simply pre-define your objects such as proteins, reactants or reactions and call the addX function
- Units will be parsed automatically if they align with the follow convention

    Unit / Unit => e.g. mole / l
    
    **!! Please make sure to separate each word or "/" via space !!**

In [ ]:
enzmldoc = EnzymeMLDocument("Your experiment name")

## User information

Information about the creators of a given EnzymeML document is stored within a list of _Creator_ objects. Via the _setCreator_ function the creator is added to the document. Note, that you can also enter a list of _Creator_ objects.

Attributes:
    - Given name
    - last name
    - E-mail

In [ ]:
creator_1 = Creator(family_name="ML", given_name="Enzyme", mail="EnzymeML@PyEnzyme")
creator_2 = Creator(family_name="Musterman", given_name="Max", mail="Max.Mustermann@PyEnzyme")

creators =  [ creator_1, creator_2 ] # for multiple creators use a list

enzmldoc.setCreator( creators )

## Vessel

- In order to add vessel information to you EnzymeML document, pre-define each by creating an instance of a _Vessel_ object.
- When finished, simply set the _Vessel_ object to the _EnzymeMLDocument_ object via its _setVessel_ method.
- Units are parsed and added to the _UnitDictionary_ by the backend.

Attributes:
    - Name: Systematic name of vessel
    - Size: Value of size
    - Unit: Size defining unit (e.g. volumetric such as 'ml')
    

In [ ]:
vessel = Vessel( 

                name="Reaction Vessel", 
                id_="v0", 
                size=1.0, 
                unit='ml' 
    
            )

In [ ]:
vessel_id = enzmldoc.setVessel(vessel)

## Proteins

- In order to add protein information to you EnzymeML document, pre-define each by creating an instance of a _Protein_ object. 
- When finished, simply add the _Protein_ object via the _addProtein_ function of the _EnzymeMLDocument_ object. 
- Units are parsed and added to the _UnitDictionary_ as well as internal IDs given by the backend.

Attributes:
    - ID: Internal identifier
    - Name: Systematic name of protein
    - Conc(entration): Value of initial concentration
    - Unit: Name of the concentration unit 
    - Sequence: Protein aminoacid sequence
    - Vessel: Name of vessel used in experiment

In [ ]:
protein_1 = Protein(
    
                    name="EnzymeMLase", 
                    sequence="ENZYMEML", 
                    vessel=vessel_id, 
                    init_conc=1.0, 
                    substanceunits="mmole / l",
                    organism="E.coli",
                    ecnumber="EC:1.2.2.4"
                    
                    )

In [ ]:
protein_1 = enzmldoc.addProtein(protein_1)

----------

**Important**

- When adding reactants the function will return the ID
- Store the ID to use it later on in a reaction 

--------

## Reactants

- In order to add reactant information to you EnzymeML document, pre-define each by creating an instance of a _Reactant_ object. 
- When finished, simply add the _Reactant_ object via the _addReactant_ function of the _EnzymeMLDocument_ object. 
- Units are parsed and added to the _UnitDictionary_ as well as internal IDs given by the backend.

Attributes:
    - Name: Systematic name of protein
    - Compartment ID: Internal ID of you pre-defined Vessel
    - Initial concentration: Value of the initial concentration
    - Substance Unit: Name of the concentration unit 
    - Constant: Whether or not the substance is at constant concentration
    
    - Inchi: String defining the INCHI-encoded molecular composition
    - Smiles: String defining the SMILES-encoded molecular composition

In [ ]:
reactant_1 = Reactant(
                        
                        name="Reactant1", 
                        vessel=vessel_id, 
                        init_conc=1.0, 
                        substanceunits="mmole / l",
                        constant=True,
                        inchi="INCHI",
                        smiles="SMILES"
    
                    )

In [ ]:
reactant_1 = enzmldoc.addReactant(reactant_1)

-------

**Important**

- When adding reactants the function will return the ID
- Store the ID to use it later on in a reaction

---------

## Reactions

- In order to add reactant information to you EnzymeML document, pre-define each by creating an instance of a _EnzymeReaction_ object. 
- When finished, simply add the _EnzymeReaction_ object via the _addReaction_ function of the _EnzymeMLDocument_ object. 
- Units are parsed and added to the _UnitDictionary_ as well as internal IDs given by the backend.


Attributes:
    
    - Name: Reaction name
    - Temperature: Value of temperature
    - Temp Unit: Unit defining the temperature
    - pH: Value of pH
    - Reversible: Whether or not the reaction is reversible
    

In [ ]:
reaction_1 = EnzymeReaction(
    
                            name="Reaction1",
                            temperature=20.0, 
                            tempunit="C", 
                            ph=7.0, 
                            reversible=True
    
                            )

### Building reaction

- In _PyEnzyme_ reactions are built by using the pre-defined reactants and protein. 
- Educts, products as well as modifiers such as a protein or buffer are added to the reaction via the respective _addXX_ method inherited by the _EnzymeReaction_ object.
- **If you previously stored reactant/protein IDs (returned by the _addXX_ function) make sure you use them when building reactions to guarantee consistency**

Attributes

    - ID: Internal ID of educt/product/modifier
    - Stoichiometry: Floating point number defining stoichiometry
    - Constant: Whether or not the participant is constant
    - enzmldoc: EnzymeMLDocument class to which it is added. Checks consistency of IDs.
    

In [ ]:
reaction_1.addEduct( 
    
                    speciesID=reactant_1, 
                    stoichiometry=1.0, 
                    isConstant=False, 
                    enzmldoc=enzmldoc 
    
                    )

reaction_1.addModifier(
    
                    speciesID=protein_1,
                    stoichiometry=1.0,
                    isConstant=True,
                    enzmldoc=enzmldoc
    
                    )

### Add replicate data

- In order to add replicates and time course data, pre-define the _Replicate_ object and set its data via the objects method _setData_.
- The replicate is then added to the respective educt/product/modifier by the given ID

Attributes

    - Replica: Unique ID for the replicate
    - Reactant: Unique ID for the reactant/protein
    - Type: Defines the type of data (e.g. concentration, photometric)
    - Data unit: Unit of given data
    - Time unit: Unit of given time
    

In [ ]:
replicate_1 = Replicate(

                        replica="Replica_1", 
                        reactant=reactant_1, 
                        type_="conc", 
                        data_unit="mmole / l", 
                        time_unit="s",
                        init_conc=1.0,
                        measurement="m0"

                    )

In [ ]:
data = [1,2,3,4,5,6] # Here should be your own data
time = [1,2,3,4,5,6] # Here should be your own data

replicate_1.setData(data, time)

In [ ]:
reaction_1.addReplicate(replicate_1, enzmldoc)

### Add reaction to _EnzymeMLDocument_

- When the creation of the reaction is completed, simply add the _EnzymeReaction_ object to the _EnzymeMLDocument_ via its _addReaction_ method.

In [ ]:
reaction1 = enzmldoc.addReaction(reaction_1)

# Modelling

- Within _PyEnzyme_ it is also possible to store kinetic models
- Create a _KineticModel_ object and add it to your reaction
- This can either be done while creating an _EnzymeReaction_ object or afterwards

Attributes:

    - Equation: String that defines the kinetic model ( use Internal IDs as variables )
    - Parameters: Dictionary with parameter name as key and respective numeric value
    

In [ ]:
equation = "s0 * vmax / ( s0 + Km )"

parameters = dict()
parameters['Km_s0'] = (1.0, "mmole / s")
parameters['vmax_s0'] = (10.0, "mmole / l")

kinetic_model = KineticModel(
    equation=equation,
    parameters=parameters,
    enzmldoc=enzmldoc 
)

In [ ]:
enzmldoc.getReaction(reaction1).setModel(kinetic_model)

# Ready to write

- Simply call the _EnzymeMLWriter_ class to write your _EnzymeMLDocument_ to an .omex container

In [ ]:
enzmldoc.printUnits()
enzmldoc.printProteins()
enzmldoc.printReactants()
enzmldoc.printReactions()

In [ ]:
out_dir = "YourDirectory/YourFilename"
EnzymeMLWriter().toFile( enzmldoc=enzmldoc, path=out_dir )